# Training 

This file contains the training part of the RNN.

This cells claims the data on Kaggle thanks to the API. 

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joebeachcapital/nsw-australia-electricity-demand-2018-2023")

print("Path to dataset files:", path)



/Users/mathisp/Code/S9/reservoir/reservoir/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/mathisp/.cache/kagglehub/datasets/joebeachcapital/nsw-australia-electricity-demand-2018-2023/versions/1


In [ ]:
import csv 
import os 
from datetime import datetime
dpath = '/Users/mathisp/Code/S9/reservoir/archive/'
data = []

#Notes Function building the data useful for the training. 
def gathering_data(csv_reader): 
    data = []
    for row in csv_reader:
        data.append(row[1:4])
    data.pop(0)
    return data 

#Doing it with each file 
def gathering_all(repo):
    lst = os.listdir("./archive/")
    lst.sort()
    # sorting to get data in the chronologic time. 
    for file in lst : 
        with open(dpath + file, mode = 'r') as file :
            csv_reader = csv.reader(file)
            data.append(gathering_data(csv_reader))

gathering_all(dpath)

# transform str into floats 
def numerical(data): 
    for elt in data :
        for time in elt:
            time[0] = datetime.strptime(time[0])
            time[1] = float(time[1])
            time[2] = float(time[2])
    return data 
num = numerical(data)
print(len(num))

[['2018/01/01 00:30:00', '6912.25', '91.86'], ['2018/01/01 01:00:00', '6745.63', '88.83'], ['2018/01/01 01:30:00', '6466.06', '73.62'], ['2018/01/01 02:00:00', '6279.8', '71.49'], ['2018/01/01 02:30:00', '6177.31', '69.27'], ['2018/01/01 03:00:00', '6050.43', '68.44'], ['2018/01/01 03:30:00', '6032.48', '69.67'], ['2018/01/01 04:00:00', '5968.5', '67.58'], ['2018/01/01 04:30:00', '5953.75', '67.09'], ['2018/01/01 05:00:00', '5962.18', '67.53'], ['2018/01/01 05:30:00', '5991.69', '65.80'], ['2018/01/01 06:00:00', '6062.03', '61.53'], ['2018/01/01 06:30:00', '6204.87', '66.62'], ['2018/01/01 07:00:00', '6352.58', '68.42'], ['2018/01/01 07:30:00', '6548.03', '69.02'], ['2018/01/01 08:00:00', '6728.17', '69'], ['2018/01/01 08:30:00', '6885.22', '68.88'], ['2018/01/01 09:00:00', '7080.7', '69.22'], ['2018/01/01 09:30:00', '7281.71', '74.04'], ['2018/01/01 10:00:00', '7474.62', '75.15'], ['2018/01/01 10:30:00', '7663.88', '75.20'], ['2018/01/01 11:00:00', '7764.89', '80.66'], ['2018/01/01 11

Building of the ESN

In [ ]:

units = 50 
leak_rate = 0.3
spectral_radius = 1.0
input_scaling = 1.0
connectivity = 0.1
input_connectivity = 0.2
regularization = 1e-8
seed = 42
X_train =  num[:30][:]
Y_train  = num[:30][2]
print(X_train)

[['2018/01/01 00:30:00', 6912.25, 91.86], ['2018/01/01 01:00:00', 6745.63, 88.83], ['2018/01/01 01:30:00', 6466.06, 73.62], ['2018/01/01 02:00:00', 6279.8, 71.49], ['2018/01/01 02:30:00', 6177.31, 69.27], ['2018/01/01 03:00:00', 6050.43, 68.44], ['2018/01/01 03:30:00', 6032.48, 69.67], ['2018/01/01 04:00:00', 5968.5, 67.58], ['2018/01/01 04:30:00', 5953.75, 67.09], ['2018/01/01 05:00:00', 5962.18, 67.53], ['2018/01/01 05:30:00', 5991.69, 65.8], ['2018/01/01 06:00:00', 6062.03, 61.53], ['2018/01/01 06:30:00', 6204.87, 66.62], ['2018/01/01 07:00:00', 6352.58, 68.42], ['2018/01/01 07:30:00', 6548.03, 69.02], ['2018/01/01 08:00:00', 6728.17, 69.0], ['2018/01/01 08:30:00', 6885.22, 68.88], ['2018/01/01 09:00:00', 7080.7, 69.22], ['2018/01/01 09:30:00', 7281.71, 74.04], ['2018/01/01 10:00:00', 7474.62, 75.15], ['2018/01/01 10:30:00', 7663.88, 75.2], ['2018/01/01 11:00:00', 7764.89, 80.66], ['2018/01/01 11:30:00', 7813.63, 77.93], ['2018/01/01 12:00:00', 7815.47, 71.48], ['2018/01/01 12:30:00

Functions allowing to build and reset the ESN. 

In [ ]:
def reset_esn():
    from reservoirpy.nodes import Reservoir, Ridge

    reservoir = Reservoir(units, input_scaling=input_scaling, sr=spectral_radius,
                          lr=leak_rate, rc_connectivity=connectivity,
                          input_connectivity=input_connectivity, seed=seed)
    readout   = Ridge(ridge=regularization, output_dim=1)

    return reservoir >> readout


In [5]:
from reservoirpy.nodes import Reservoir, Ridge

reservoir = Reservoir(units, input_scaling=input_scaling, sr=spectral_radius,
                      lr=leak_rate, rc_connectivity=connectivity,
                      input_connectivity=input_connectivity, seed=seed)

readout   = Ridge(ridge=regularization, output_dim=1)

esn = reservoir >> readout